This is the second part of project. Please refer to part 1 for information on how the dataset was collected and cleaned. 

In [ ]:
# Optional - causes longer run times
# The data frame that will be used has 120 columns to start with and I want to be able to see all of them
pd.set_option("display.max_columns", None)

# Also, to see all rows
pd.set_option('display.max_rows', None)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Initial Feature Selection & Data Transformation

As shown below, this current dataset has 1,162 observations and 115 (numerical) features currently, so there is a further need to preprocess the data further. Here is what the following process will look like:

- For some initial feature selection, I will employ a basic filter method: eliminate all redundant features as defined by having a Pearson's Correlation Coefficient value of over 0.98 (absolute value). This value is set semi-arbitrarily high because the purpose is to eliminate features that are completely explained (correlation coefficient = 1/-1) but some of the rate statistics have been rounded, so I'm using the value 0.98.

- Next, an examination of the distributions of features will be done to determine whether there should be any transformations done to highly-skewed data.

In [ ]:
# Read in data
df = pd.read_csv('NBA_FINAL_DATA.csv', index_col=0)

df

In [ ]:
display(df.info())

display(df.shape)

In [ ]:
# Check for nulls
df.isnull().sum()

In [ ]:
# Check for duplicate rows
df.duplicated().sum()

The data frame has 1162 observations and 120 features. The primary concern here is the curse of dimensionality; this is going to be address later with dimensionality reduction. However, before moving on to that step, I want to look at and study the dataset first. Before moving forward, I want to instantiate the features variable, X, with only numerical columns (also without age, games played).

In [ ]:
X = df.drop(['PLAYER', 'TEAM', 'AGE', 'GP', 'POSITION'], axis=1)

X.info()

Having 115 features is definitely going to cause any sort of correlation heatmap to be an inefficient way of looking at heavily correlated features. However, I will create one just for visualization purposes which will give us a visual, high-level overview of what the correlation coefficients between the features look like where the absolute value of the correlation coefficients are greater than 0.98.

In [ ]:
plt.figure(figsize=(60,40))
mask = np.triu(np.ones_like(X.corr()))
sns.heatmap(X.corr()[abs(X.corr())>=.98], mask=mask, vmin=-1, vmax=1, cmap='coolwarm')
plt.title('Correlation Matrix of All Features')
plt.show()

From the visual above, there doesn't seem to be an overly large amount of redundant features (as defined by correlation > 0.98 mentioned above). I will take a look at all the columns and then select features to drop; of the pair of features who shared 0.98 correlation, I will personally decide which feature to drop. This decision will be made based off interpretability; for example (as seen below), between >=24FT_FGA and 3PA, I will drop the former because the latter name is more interpretable and used more often.

In [ ]:
corr = X.corr()

corr.shape

corr.abs().unstack().sort_values(ascending=False).drop_duplicates().head(50)

In [ ]:
# Dropping features 
X = X.drop(['%FGA_2PT', 'FGM_%AST', '2FGM_%AST', '%LOOSE_RECOVERED_OFF', '>=24FT_FGA', '>=24FT_FGM', 'SCREEN_AST_PTS', 
            '%TEAM_DREB', '%TEAM_REB', 'FGM', 'FTM', '3PM', '%TEAM_AST', '<8FT_FGM', '8-16_FGM'], axis=1)

# Check number of features left
X.shape

Next, I will check the distributions of the features and determine whether transformation should be applied. To determine whether a feature is heavily skewed, I will utilize a threshold: if abs(mean) >= abs(1.25 * median), then heavily skewed to the right and if abs(mean) <= abs(0.75 * median), then heavily skewed to the left. Before I check, I want to visualize the distributions for all features as well.

In [ ]:
# Function code modified from one provided by BrainStation
def hist_create(df,column,transform = lambda x: x):

    plt.figure()
    plt.hist(transform(df[column]))
    plt.title(f"histogram of {column}")
    plt.show()

In [ ]:
for column in X.columns:
    
    hist_create(X,column)

In [ ]:
right_skewed = [column for column in X.columns if X[column].mean() >= 1.25*X[column].median()]

print(f'Number of Right Skewed Features: {len(right_skewed)}')

print(f'Right Skewed Columns: {right_skewed}')

In [ ]:
not_right_skewed = [column for column in X.columns if column not in right_skewed]
print(f'Number of non-right skewed features: {len(not_right_skewed)}')

For right-skewed data, I will consider applying log transformation. Before applying any actual transformation, using the create_hist function from earlier, visualize what a log transformation will look like for the features that are heavily right skewed.

In [ ]:
for column in right_skewed:
    hist_create(X,column,lambda x: np.log(x + 1))

In [ ]:
# Apply transformation
for column in right_skewed:
    X[column] = np.log(X[column] + 1)

In [ ]:
left_skewed = [column for column in not_right_skewed if abs(X[column].mean()) <= abs((.75*X[column].median()))]
len(left_skewed)

# Validating Clustering Tendencies of Dataset

Before moving on to the steps of dimensionality reduction and actual modelling, I wanted to perform a few quick checks to ensure that the dataset has clustering tendencies. These checks will be done once optimal number of clusters and other parameters are optimized as well but will also be done now without improving our dataset to get a baseline snapshot of our current dataset. There will be two methods to determine clustering tendencies:

1) The first will be using visual heuristics - I will use PCA and TSNE to reduce the dimensionality of the dataset to 2 components purely for visualization purposes and perform a visual check to see if there are patterns that exhibit clustering tendencies.

2) Afterwards, using the pyclustertend library, I will calculate the hopkins score - a metric that explains spatial randomness of the data. The pyclustertend library documentation as well as other online sources indicate that a score around 0.5 expresses non-clustering tendencies (data is uniformly distributed) and scores that tend to 0 indicate high clustering tendencies. Using the Pyclustertend library as well as other online sources, the threshold set will be 3.0.

### Visualizing in 2D with PCA

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
# Scale data
mm_scaler = MinMaxScaler()
s_scaler = StandardScaler()
r_scaler = RobustScaler()

X_mm = mm_scaler.fit_transform(X)
X_ss = s_scaler.fit_transform(X)
X_rs = r_scaler.fit_transform(X)

In [ ]:
# PCA
pca = PCA(n_components=2)

# fit_transform pca and print explained variance ratio sum
X_mm_pca = pca.fit_transform(X_mm)
print(f'Explained Variance Ratio for X_mm: {pca.explained_variance_ratio_.sum()}')

X_ss_pca = pca.fit_transform(X_ss)
print(f'Explained Variance Ratio for X_ss: {pca.explained_variance_ratio_.sum()}')

X_rs_pca = pca.fit_transform(X_rs)
print(f'Explained Variance Ratio for X_rs: {pca.explained_variance_ratio_.sum()}')

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, figsize=(12,20))
fig.suptitle('PCA: 2 Dimensional Visualization of NBA Dataset')
ax1.scatter(X_mm_pca[:,0], X_mm_pca[:,1])
ax1.set_title('MinMax Scaled')

ax2.scatter(X_ss_pca[:,0], X_ss_pca[:,1])
ax2.set_title('Standard Scaled')

ax3.scatter(X_rs_pca[:,0], X_rs_pca[:,1])
ax3.set_title('Robust Scaled')

### Visualizing in 2D with TSNE

In [ ]:
tsne = TSNE(n_components=2, random_state=14)

X_mm_tsne = tsne.fit_transform(X_mm)
X_ss_tsne = tsne.fit_transform(X_ss)
X_rs_tsne = tsne.fit_transform(X_rs)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, figsize=(12,20))
fig.suptitle('TSNE: 2 Dimensional Visualization of NBA Dataset')
ax1.scatter(X_mm_tsne[:,0], X_mm_tsne[:,1])
ax1.set_title('MinMax Scaled')

ax2.scatter(X_ss_tsne[:,0], X_ss_tsne[:,1])
ax2.set_title('Standard Scaled')

ax3.scatter(X_rs_tsne[:,0], X_rs_tsne[:,1])
ax3.set_title('Robust Scaled')

From the visuals above, the clustering tendencies of the datasets do appear a bit low. However, that being said, for the TSNE visuals, the minmax-scaled data and the robust-scaled data clearly exhibit at least 2-3 distinct clusters and one could make the argument that the standard-scaled data shows 3 distinct clusters.

The next check will be with the hopkins test as mentioned earlier using a threshold of 0.3.

In [ ]:
from pyclustertend import hopkins

print(f'Hopkins Test Score For X_mm: {hopkins(X_mm, X_mm.shape[0])}')
print(f'Hopkins Test Score For X_ss: {hopkins(X_ss, X_ss.shape[0])}')
print(f'Hopkins Test Score For X_rs: {hopkins(X_rs, X_rs.shape[0])}')

The hopkins test scores for all three differently-scaled datasets - using the threshold of 0.3 - indicates that the dataset does exhibit clustering tendencies (the test scores are all relatively similar).

# Dimensionality Reduction

Even after I manually removed redundant features earlier (using threshold of 0.98 correlation coefficient), the dataset contains 100 features. As a result, the curse of dimensionality is a big concern that needs to be addressed before clustering. 

Ideally, I would want to apply feature selection first in order to narrow the amount of features to the most important ones. That being said, feature selection for unsupervised learning is very difficult to do and can inject a lot of bias into the dataset. I have split this notebook/project into two different sections:

1) The first section (the one that will be performed now) is a more conservative process. I will undergo the regular clustering processes by scaling the dataset, applying dimensionality reduction only, and then clustering.

2) For the second section (after first section is complete), I will again perform scaling on the dataset, applying feature selection (through PFA), then applying dimensionality reduction, and then clustering.

# PCA
There are two different dimensionality reduction processes that I will perform. The first is PCA and in order to choose the optimal number of PCs, I will use an elbow plot and check the optimal number of PCs for the three, differently-scaled datasets.

In [ ]:
# Minmax scaled data
pca = PCA()

pca_mm = pca.fit(X_mm)

# Instantiate variable for explained variance ratio (minmax scaled dataset)
expl_var_mm = pca_mm.explained_variance_ratio_

# Plot scree (eblow) plot
plt.figure(figsize=(12, 6))
plt.plot(range(1,101),expl_var_mm,marker='.')
plt.title('Proportion of Variance Explained for Number of PCs')
plt.xlabel('Number of PCs')
plt.ylabel('Proportion of Variance Explained')
plt.xticks(range(1,101,2))
plt.grid()
plt.show()

In [ ]:
# Check how much variance can be explained with 5 components
np.cumsum(expl_var_mm[:5])

With the elbow plot, the optimal number of components was shown to be 5 components that explained about 70% of the total variance in the dataset. I also want to quickly evaluate how many additional components needed to increase this explained variance ratio significantly.

Arbitrarily selecting 10% as a measure of significant increase in explained variance, I will plot of cumulative sum graph to see how many additional components would be needed to get an explained variance ratio of 80%.

In [ ]:
# Plot out the cumulative sum graph
plt.figure(figsize=(12,6))
plt.plot(range(1,101), np.cumsum(expl_var_mm), marker='.')
plt.axhline(0.8, c='r', linestyle='--')
plt.title('Cumulative Sum of Explained Variance Against Number of PCs')
plt.xlabel('Number of PCs')
plt.ylabel('Cumulative Sum of Explained Variance')
plt.xticks(range(1,101,2))
plt.grid()
plt.show()

The graph indicates that 10 more components would be needed (100% increase) in order to increase the explained variance by 10%. I will choose to stick with n_components=5 which provides enough explained variance while maintaining good interpretability. I will continue these steps for the standard-scaled and robust-scaled datasets.

In [ ]:
# Standard scaled data
pca = PCA()
pca_ss = pca.fit(X_ss)

expl_var_ss = pca_ss.explained_variance_ratio_

# Plot scree (eblow) plot
plt.figure(figsize=(12, 6))
plt.plot(range(1,101),expl_var_ss,marker='.')
plt.title('Proportion of Variance Explained for Number of PCs')
plt.xlabel('Number of PCs')
plt.ylabel('Proportion of Variance Explained')
plt.xticks(range(1,101,2))
plt.grid()
plt.show()

In [ ]:
print(f'Explained Variance for 5 Components: {np.cumsum(expl_var_ss[:5])}')

In [ ]:
# Robust-scaled data
pca = PCA()
pca_rs = pca.fit(X_rs)

expl_var_rs = pca_rs.explained_variance_ratio_

# Plot scree (eblow) plot
plt.figure(figsize=(12, 6))
plt.plot(range(1,101),expl_var_rs,marker='.')
plt.title('Proportion of Variance Explained for Number of PCs')
plt.xlabel('Number of PCs')
plt.ylabel('Proportion of Variance Explained')
plt.xticks(range(1,101,2))
plt.grid()
plt.show()

In [ ]:
print(f'Explained Variance for 8 Components: {np.cumsum(expl_var_rs[:8])}')

For the standard scaled dataset, I select 5 components (explained variance of 0.65) and for the robust scaled dataset, I selected 8 components (explained variance of 0.7).

Summary:
- minmax data: 5 components
- standard data: 5 components
- robust data: 8 components

In [ ]:
# Transform all datasets
X_mm_pca = pca_mm.transform(X_mm)
X_ss_pca = pca_ss.transform(X_ss)
X_rs_pca = pca_rs.transform(X_rs)

The next step will be to perform the actual clustering with these finalized datasets. I will evaluate the clustering performance using inertia and silhouette scores. These are the two most common metrics implemented in the sci-kit library used for unsupervised learning when ground truth labels are unknown. For KMeans hyperparameters, I am semi-arbitrarily selecting init='k-means++' to ensure centers are are initialized with some distance apart, and increasing n_init and max_iter to find a more stable solution (balancing with computational costs)

### MinMax Data

In [ ]:
from sklearn.cluster import KMeans

# Get inertia values as a function of different cluster sizes
ks = np.arange(1,15)

inertias = []

for k in ks:
    
    kmeans = KMeans(n_clusters=k, random_state=12, init='k-means++', n_init=50, max_iter=500)
    kmeans.fit(X_mm_pca)
    
    inertias.append(kmeans.inertia_)
    
# Plot inertias against k values (number of clusters)    
plt.figure(figsize=(12, 6))
plt.plot(ks, inertias, marker='o')
plt.grid()
plt.xlabel('K values')
plt.ylabel('inertia')
plt.title('K Means: Number of Clusters (k) vs Inertia')
plt.show()

In [ ]:
from sklearn.metrics import silhouette_score
ks = np.arange(2,15)

sil_scores = []

for k in ks:
    
    kmeans = KMeans(n_clusters=k, random_state=12, init='k-means++', n_init=50, max_iter=500)
    kmeans.fit(X_mm_pca)
    
    sil_scores.append(silhouette_score(X_mm_pca, kmeans.labels_))

# Plot sil_scores against number of clusters
plt.figure(figsize=(12, 6))
plt.plot(ks, sil_scores, marker='o')
plt.grid()
plt.xlabel('K values')
plt.ylabel('sil_scores')
plt.title('K Means: Number of Clusters (k) vs Silhouette Scores')
plt.savefig(fname='C:/Users/imdan/downloads/data/minmax_silscore.jpeg')

For the minmax dataset, both metrics indicate that the optimal number of clusters is 3 the a silhouette score of about 0.21. I will perform the same steps for the other datasets before moving on.

### Standard Data

In [ ]:
# Get inertia values as a function of different cluster sizes
ks = np.arange(1,15)

inertias = []

for k in ks:
    
    kmeans = KMeans(n_clusters=k, random_state=12, init='k-means++', n_init=50, max_iter=500)
    kmeans.fit(X_ss_pca)
    
    inertias.append(kmeans.inertia_)
    
# Plot inertias against k values (number of clusters)    
plt.figure(figsize=(12, 6))
plt.plot(ks, inertias, marker='o')
plt.grid()
plt.xlabel('K values')
plt.ylabel('inertia')
plt.title('K Means: Number of Clusters (k) vs Inertia')
plt.show()

In [ ]:
ks = np.arange(2,15)

sil_scores = []

for k in ks:
    
    kmeans = KMeans(n_clusters=k, random_state=12, init='k-means++', n_init=50, max_iter=500)
    kmeans.fit(X_ss_pca)
    
    sil_scores.append(silhouette_score(X_ss_pca, kmeans.labels_))

# Plot sil_scores against number of clusters
plt.figure(figsize=(12, 6))
plt.plot(ks, sil_scores, marker='o')
plt.grid()
plt.xlabel('K values')
plt.ylabel('sil_scores')
plt.title('K Means: Number of Clusters (k) vs Silhouette Scores')
plt.show()

### Robust Dataset

In [ ]:
ks = np.arange(1,15)

inertias = []

for k in ks:
    
    kmeans = KMeans(n_clusters=k, random_state=12, init='k-means++', n_init=50, max_iter=500)
    kmeans.fit(X_rs_pca)
    
    inertias.append(kmeans.inertia_)
    
# Plot inertias against k values (number of clusters)    
plt.figure(figsize=(12, 6))
plt.plot(ks, inertias, marker='o')
plt.grid()
plt.xlabel('K values')
plt.ylabel('inertia')
plt.title('K Means: Number of Clusters (k) vs Inertia')
plt.show()

In [ ]:
ks = np.arange(2,15)

sil_scores = []

for k in ks:
    
    kmeans = KMeans(n_clusters=k, random_state=12, init='k-means++', n_init=50, max_iter=500)
    kmeans.fit(X_rs_pca)
    
    sil_scores.append(silhouette_score(X_rs_pca, kmeans.labels_))

# Plot sil_scores against number of clusters
plt.figure(figsize=(12, 6))
plt.plot(ks, sil_scores, marker='o')
plt.grid()
plt.xlabel('K values')
plt.ylabel('sil_scores')
plt.title('K Means: Number of Clusters (k) vs Silhouette Scores')
plt.show()

From the visuals above, the optimal number of clusters for all datasets was 3. However, the minmax dataset provided the highest silhouette score with 0.21. This silhouette score is quite low and indicates that I should try to increase this score before moving forward. There are two steps to increase the silhouette score for the KMeans model:

1) I can play around with the number of components for PCA-transformed data and add additional components to achieve a higher explained variance ratio.

2) I can try a different type of dimensionality reduction.

I will proceed with the latter option (reasoning is the same as to why I didn't increase the explained variance ratio earlier). For now, I will implement dimensionality reduction using Linear Discriminant Analysis. This makes sense because LDA uses class labels to find a subspace that maximizes class seperability. I will apply LDA with the classes before the NBA players' traditional positions. For LDA, scaling has no effect, so I will only apply LDA on the minmax data.

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# Instatiate target (clases) as the traditional positions
y = df['POSITION']

lda = LDA()
lda_mm = lda.fit(X_mm, y)


# Instantiate variable for explained variance ratio (minmax scaled dataset)
expl_var_mm = lda_mm.explained_variance_ratio_

print(expl_var_mm)

LDA captured about 95% of the total variance in just two components. 

In [ ]:
# Use 2 components
lda = LDA(n_components=2)
X_lda = lda.fit_transform(X_mm, y)

plt.figure(figsize=(12,8))
plt.scatter(X_lda[:,0], X_lda[:,1])
plt.title('LDA: 2 Dimensional Visualization of NBA Dataset')
plt.show()

In [ ]:
ks = np.arange(1,15)

inertias = []

for k in ks:
    
    kmeans = KMeans(n_clusters=k, random_state=12, init='k-means++', n_init=50, max_iter=500)
    kmeans.fit(X_lda)
    
    inertias.append(kmeans.inertia_)
    
# Plot inertias against k values (number of clusters)    
plt.figure(figsize=(12, 6))
plt.plot(ks, inertias, marker='o')
plt.grid()
plt.xlabel('K values')
plt.ylabel('inertia')
plt.title('K Means: Number of Clusters (k) vs Inertia')
plt.show()

In [ ]:
ks = np.arange(2,15)

sil_scores = []

for k in ks:
    
    kmeans = KMeans(n_clusters=k, random_state=10, init='k-means++', n_init=50, max_iter=500)
    kmeans.fit(X_lda)
    
    sil_scores.append(silhouette_score(X_lda, kmeans.labels_))

# Plot sil_scores against number of clusters
plt.figure(figsize=(12, 6))
plt.plot(ks, sil_scores, marker='o')
plt.grid()
plt.xlabel('K values')
plt.ylabel('sil_scores')
plt.title('K Means: Number of Clusters (k) vs Silhouette Scores')
plt.savefig(fname='C:/Users/imdan/downloads/data/lda_silscore.jpeg')

The silhouette score here is significantly better than our PCA-transformed dataset. Additionally, the visualization definitely showcases higher clustering tendencies than with our PCA/TSNE visualizations. However, there is a big issue here indicated that LDA is able to capture 95% of the total variance with just the first 2 components. LDA is well-known to be prone to overfitting and that is what may be happening here. To address this issue, I will first play around with reducing the number of features before even applying LDA dimensionality reduction.

# Additional Feature Selection & Dimensionality Reduction

I will be performing additional filter methods for feature selection to reduce the number of features:

1) Variance threshold to eliminate constant columns (might be 0 but will check)

2) Using Pearson's Correlation Coefficient and eliminating features that share a coefficient value of over 92% and I will be manually dropping these to keep the more interpretable features.

In [ ]:
# Normalize before applying variance threshold

X_mm = mm_scaler.fit_transform(X)

X_mm = pd.DataFrame(data=X_mm, columns=X.columns)

In [ ]:
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold()
practice = selector.fit_transform(X_mm)

In [ ]:
practice.shape

No columns were dropped by eliminating constant features - moving on to filtering out using correlation coefficient.

In [ ]:
corr = X.corr()

corr.shape

corr.abs().unstack().sort_values(ascending=False).drop_duplicates().head(50)

In [ ]:
X = X.drop(['%TEAM_OREB', 'FGA', '%TEAM_3PA', '%FGA_3PT', '<8FT_FGA', '%TEAM_FTA', '%TEAM_FGA', 'DREB', 'FTA', 
            '16-24_FGA', '%TEAM_FGM', '%FGA_3PT', '%PTS_3PT', 'DREB%', '%TEAM_FTA', 'eFG%', 'CONTEST_SHOTS', 
            'OPP_PAINT_PTS', '%PTS_2PT', 'DEFLECTIONS', '%TEAM_FTM', '>=24FT_FG%'], axis=1)

In [ ]:
# Check how many features remaining
X.shape

We have 80 features after removing the heavily correlated features. This is still a large number of features but I will apply dimensionality reduction moving forward. From previously, it's clear that applying LDA (with the traditional 5 positions being the labels) provided a much higher silhouette score. However, the problem was that there seemed to be overfitting because the first 2 components accounts for about 95% of the variation. 

I will be moving forward with the LDA dataset; however, I need to ensure that the explained variance isn't so high because it may adversely affect clustering algorithms. Additionally, from this step forward, I will only be using minmax-scaled data as minmax-scaler seems to be the most common scaler for clustering and minmax makes a lot of sense when comparing statistics for NBA players.

In [ ]:
# Scale
mm_scaler = MinMaxScaler()
X_mm = mm_scaler.fit_transform(X) 

In [ ]:
# Try the same method as previously with new dataset containing 80 features instead of 100
lda = LDA()
X_lda = lda.fit(X_mm, y)

expl_var_lda = X_lda.explained_variance_ratio_

print(expl_var_lda)

It's clear that even with the new dataset, there appears to be overfitting with an explained variance ratio of over 0.95 with just the first two components. In order to ensure that overfitting isn't a problem moving forward, I will be using LDA with the 'eigen' solver that allows a shrinkage parameter. The optimal shrinkage parameter will be determined somewhat arbitrarily; I will select the parameter that provides around 0.8-0.85 explained variance in the first 2 components.

In [ ]:
shrink_vals = [0.009, 0.008, 0.007, 0.006, 0.005, 0.004, 0.003, 0.002, 0.001]

for val in shrink_vals:
    lda = LDA(solver='eigen', shrinkage=val, n_components=2)
    X_lda = lda.fit(X_mm, y)
    
    explained_var = X_lda.explained_variance_ratio_
    
    if (explained_var[0] + explained_var[1]) >= 0.7:
        print(f'Shrinkage: {val}', f'Component 1: {explained_var[0]}', f'Component 2: {explained_var[1]}')

I will be selecting the shrinkage value to be 0.002 which provides about 84% explained variance in the first 2 components.

In [ ]:
# Apply dimensionality reduction to dataset
lda = LDA(solver='eigen', shrinkage=0.002, n_components=2)

X_lda = lda.fit_transform(X_mm, y)

print(X_lda.shape)
print(lda.explained_variance_ratio_)

In [ ]:
# Plot
plt.figure(figsize=(12,6))
plt.scatter(X_lda[:,0], X_lda[:,1], c='pink')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.title('LDA: 2 Dimensional Visualization of NBA Dataset')
plt.savefig(fname='C:/Users/imdan/downloads/data/lda_featurespace.jpeg')

# Clustering Models

I will start to perform some clustering and analysis of the clusters. The optimal number of clusters will be selected by examining inertia, silhouette scores, and davies-bouldin scores.

### KMeans

In [ ]:
# Plot Inertias
ks = np.arange(1,15)

inertias = []

for k in ks:
    
    kmeans = KMeans(n_clusters=k, random_state=12, init='k-means++', n_init=50, max_iter=500)
    kmeans.fit(X_lda)
    
    inertias.append(kmeans.inertia_)
    
# Plot inertias against k values (number of clusters)    
plt.figure(figsize=(12, 6))
plt.plot(ks, inertias, marker='o')
plt.grid()
plt.xlabel('K values')
plt.xlim(1, 14.5)
plt.axvline(x=3, c='red')
plt.ylabel('inertia')
plt.title('K Means: Number of Clusters (k) vs Inertia')
plt.savefig(fname='C:/Users/imdan/downloads/data/kmeans_intertia.jpeg')

In [ ]:
# Plot Silhouette Scores
ks = np.arange(2,15)

sil_scores = []

for k in ks:
    
    kmeans = KMeans(n_clusters=k, random_state=12, init='k-means++', n_init=50, max_iter=500)
    kmeans.fit(X_lda)
    
    sil_scores.append(silhouette_score(X_lda, kmeans.labels_))

# Plot sil_scores against number of clusters
plt.figure(figsize=(12, 6))
plt.plot(ks, sil_scores, marker='o')
plt.grid()
plt.xlabel('K values')
plt.ylabel('sil_scores')
plt.title('K Means: Number of Clusters (k) vs Silhouette Scores')
plt.show()

In [ ]:
from sklearn.metrics import davies_bouldin_score

ks = np.arange(2,15)

db_scores = []

for k in ks:
    
    kmeans = KMeans(n_clusters=k, random_state=12, init='k-means++', n_init=50, max_iter=500)
    kmeans.fit(X_lda)
    
    db_scores.append(davies_bouldin_score(X_lda, kmeans.labels_))

# Plot sil_scores against number of clusters
plt.figure(figsize=(12, 6))
plt.plot(ks, db_scores, marker='o')
plt.grid()
plt.xlabel('K values')
plt.ylabel('DB_scores')
plt.title('K Means: Number of Clusters (k) vs Davies Bouldin Scores')
plt.show()

In [ ]:
ks = np.arange(2,15)
plt.style.use('default')
plt.figure(figsize=(12,6))
plt.plot(ks, db_scores, marker='o', label='DBI score')
plt.plot(ks, sil_scores, marker='o', label='silhouette score')
plt.grid()
plt.xlabel('K: Number of Clusters')
plt.ylabel('Performance Metrics')
plt.xlim(1, 14.5)
plt.title('K Means: Number of Clusters (k) vs Metrics')
plt.axvline(x=3, c='red')
plt.legend()
plt.savefig(fname='C:/Users/imdan/downloads/data/kmeans_metrics1.jpeg')

Using all the metrics above, I will be selecting 3 clusters as the optimal number of clusters to examine initially.

In [ ]:
# Get cluster labels
kmeans = KMeans(n_clusters=3, random_state=12, init='k-means++', n_init=50, max_iter=500)
kmeans.fit(X_lda)
kmeans_labels = kmeans.labels_

In [ ]:
# Plot
plt.figure(figsize=(12,8))
plt.style.use('dark_background')
plt.scatter(X_lda[:,0], X_lda[:,1], c=kmeans_labels, cmap='gist_rainbow')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.title('LDA: NBA Players Clusters 2020-2022')
plt.savefig(fname='C:/Users/imdan/downloads/data/3clusters1.jpeg')

This definitely does look like the most naturally separated areas visually. I will analyze the information further by checking the variance within the statistics for each feature.

In [ ]:
X_mm = mm_scaler.fit_transform(X)
X_mm = pd.DataFrame(data=X_mm, columns = X.columns)

In [ ]:
kmeans3_df = X_mm.copy()

# Create labels column
kmeans3_df['kmeans_labels'] = kmeans_labels

# Check clusters' statistical means
grouped_kmeans3 = kmeans3_df.groupby('kmeans_labels', as_index=False).mean().T

In [ ]:
grouped_kmeans3

In [ ]:
grouped_kmeans3.columns = ['Cluster 0', 'Cluster 1', 'Cluster 2']

In [ ]:
grouped_kmeans3['variance'] = grouped_kmeans3.var(axis=1)

grouped_kmeans3.sort_values(by='variance', ascending=False).head(20)

In [ ]:
# Finding features with highest variance between clusters
grouped_kmeans3_sorted = grouped_kmeans3.sort_values(by='variance', ascending=False).head(20)

# creating separate table
grouped_kmeans3_sorted = grouped_kmeans3_sorted.drop('kmeans_labels', axis=0)

# Check
grouped_kmeans3_sorted

In order to get a better sense of the variability of features between the clusters, a graph will be created to show the features with highest variance between clusters. However, it is important to keep in mind that this isn't a way of measuring 'feature importance' - in other words, which features were the most important to determining clusters that players belonged to. Feature importance will be explored later on.

In [ ]:
# set plot style: grey grid in the background:
sns.set(style="darkgrid")

plt.figure(figsize=(15,8))
sns.barplot(x=grouped_kmeans3_sorted['variance'], y=grouped_kmeans3_sorted.index)
plt.title('Top 20 Features by Between-Cluster Variance')
plt.xlabel('Variance')
plt.ylabel('Features')
plt.savefig(fname='C:/Users/imdan/downloads/data/report_01.jpeg')

In [ ]:
df_kmeans3 = df.copy()
df_kmeans3['kmeans_label'] = kmeans_labels

In [ ]:
df_kmeans3.loc[df_kmeans3['kmeans_label'] == 0].head(20)

In [ ]:
df_kmeans3.loc[df_kmeans3['kmeans_label'] == 1].head(20)

In [ ]:
df_kmeans3.loc[df_kmeans3['kmeans_label'] == 2].head(20)

In [ ]:
df_kmeans3.loc[df_kmeans3['kmeans_label'] == 0, 'kmeans_label'] = 'Cluster 0'
df_kmeans3.loc[df_kmeans3['kmeans_label'] == 1, 'kmeans_label'] = 'Cluster 1'
df_kmeans3.loc[df_kmeans3['kmeans_label'] == 2, 'kmeans_label'] = 'Cluster 2'

# For visuals
df_kmeans3.to_csv('C:/Users/imdan/downloads/data/df_kmeans3.csv')

In [ ]:
df_kmeans3[['POSITION', 'kmeans_label']].head(10)

In [ ]:
df_kmeans3_ = df_kmeans3.copy()

In [ ]:
df_kmeans3_.loc[df_kmeans3['POSITION'] == 'PG', 'POSITION'] = 'Guard'
df_kmeans3_.loc[df_kmeans3['POSITION'] == 'SG', 'POSITION'] = 'Guard'
df_kmeans3_.loc[df_kmeans3['POSITION'] == 'SF', 'POSITION'] = 'Forward'
df_kmeans3_.loc[df_kmeans3['POSITION'] == 'PF', 'POSITION'] = 'Forward'
df_kmeans3_.loc[df_kmeans3['POSITION'] == 'C', 'POSITION'] = 'Center'

In [ ]:
# Plot counts of clusters labels by positions
plt.figure(figsize=(10,6))
plt.style.use('seaborn-pastel')
plt.title('Number of Players per Cluster (3)')
plt.xlabel('NBA Position')
plt.ylabel('Number of Players')
sns.countplot(data=df_kmeans3_, x='POSITION', hue='kmeans_label').set(xlabel='NBA Position', ylabel='Number of Players')
plt.savefig(fname='C:/Users/imdan/downloads/data/3clusters.jpeg', facecolor='lightcyan')

In [ ]:
# Plot counts of clusters labels by positions
plt.figure(figsize=(8,6))
plt.title('Counts of Cluster Labels by Traditional Positions')
sns.countplot(data=df_kmeans3, x='POSITION', hue='kmeans_label')
plt.show()

From above, it's clear that for the most part, the clusters are separated similarly to how the traditional positions were separated (mostly by size). Cluster 0 indicated players who are smaller and play like guards (PGs and SGs); Cluster 1 reflects mid-sized wing players (SGs, SFs, PFs) and Cluster 2 highlights bigs (PFs and Cs). The separation is quite clear no Centers (C) in cluster 0.

This was to be expected and not surprising at all. Intuitively, it's clear that players of different sizes are going to show the clearest separation because a player who is a certain size cannot do things like players of a different size.

The next step is where the real analysis begins. Similar to other unsupervised ML projects, because there are no ground truth labels, there are also no true measures of the optimal numbers of clusters. For this project, I will refer back to my business problem and the value that this project will add: identify/sub-segment players into roles that are more clearly defined than the current 5 traditional positions. Consequently, I believe that to add more value, a larger number of clusters is going to be more valuable. Referring back to the graph of silhouette scores earlier, I will proceed with analyzing 7 separate clusters. The reason is that 7 provides me with a high number of segments while also keeping the silhouette score at around 0.38 which is generally considered to indicate good separation.

In [ ]:
# 7 clusters
kmeans = KMeans(n_clusters=7, random_state=1, init='k-means++', n_init=100, max_iter=500)
kmeans.fit(X_lda)
kmeans_labels = kmeans.labels_

In [ ]:
# Plot
plt.figure(figsize=(12,6))
plt.style.use('dark_background')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.scatter(X_lda[:,0], X_lda[:,1], c=kmeans_labels, cmap='rainbow')
plt.title('Final KMeans: NBA Players Clusters 2020-2022')
plt.savefig(fname='C:/Users/imdan/downloads/data/7clusters0.jpeg')

plt.show()

In [ ]:
kmeans7_df = X_mm.copy()

# Create labels column
kmeans7_df['kmeans_labels'] = kmeans_labels

# Check clusters' statistical means
grouped_kmeans7 = kmeans7_df.groupby('kmeans_labels', as_index=False).mean().T

In [ ]:
grouped_kmeans7

In [ ]:
grouped_kmeans7.columns = ['Cluster 0', 'Cluster 1', 'Cluster 2', 'Cluster 3', 'Cluster 4', 'Cluster 5', 
                           'Cluster 6']

grouped_kmeans7['variance'] = grouped_kmeans7.var(axis=1)

grouped_kmeans7_sorted = grouped_kmeans7.drop('kmeans_labels', axis=0).sort_values(by='variance', ascending=False).head(20)

In [ ]:
# set plot style: grey grid in the background:
sns.set(style="darkgrid")

plt.figure(figsize=(10,8))
sns.barplot(x=grouped_kmeans7_sorted['variance'], y=grouped_kmeans7_sorted.index)
plt.title('Top 20 Features by Between-Cluster Variance: 7 Clusters')
plt.xlabel('Variance')
plt.ylabel('Features')
plt.savefig(fname='C:/Users/imdan/downloads/data/report_1.jpeg')

In [ ]:
# Create rankings
ranked_kmeans7 = kmeans7_df.groupby('kmeans_labels', as_index=False).mean().rank(method='max').T

ranked_kmeans7.columns = ['Cluster 0' , 'Cluster 1', 'Cluster 2', 'Cluster 3', 'Cluster 4', 'Cluster 5', 'Cluster 6']

ranked_kmeans7 = ranked_kmeans7.drop('kmeans_labels', axis=0)

ranked_kmeans7

# Feature Importance

Before moving on to analyzing the individual clusters separately, I want to explore which features are the most important when determining these clusters. A big shortcoming of the KMeans algorithm is that all the feature dimensions are weighed equally. In order to get an idea of which features are more relevant when determining the 7 found clusters, I will be training a random forest model with the cluster labels as the target variable. 

This random forest model - provided that a decent accuracy score is achieved - can tell us which features were the most important or 'predictive.'

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn import set_config

# Initialize estimator - random forest classifier
estimators = [('normalise', MinMaxScaler()),
              ('rfc', RandomForestClassifier())]

# Initialize pipeline
pipe = Pipeline(estimators)

In [ ]:
# Initial gridsearchCV to test for best model hyperparameters
from sklearn.model_selection import GridSearchCV
from tempfile import mkdtemp
cachedir = mkdtemp()

pipe = Pipeline(estimators, memory = cachedir)

params = {'normalise': [MinMaxScaler()],
          'rfc__n_estimators': [25, 50, 75, 100, 125, 150, 175, 200],
          'rfc__max_depth': [3, 5, 7, 9], 
          'rfc__min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8],
          'rfc__max_features': ['sqrt', 'log2', None, 0.2]}

grid_search = GridSearchCV(pipe, param_grid=params)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=12)
    
fitted_search = grid_search.fit(X_train, y_train)

In [ ]:
# Best hyperparameter values
fitted_search.best_params_

In [ ]:
# Best score
fitted_search.best_score_

In [ ]:
# Gridsearch 2
estimators = [('normalise', MinMaxScaler()),
              ('rfc', RandomForestClassifier(max_depth=9))]

pipe = Pipeline(estimators)

cachedir = mkdtemp()

pipe = Pipeline(estimators, memory = cachedir)

params = {'normalise': [MinMaxScaler()],
          'rfc__n_estimators': [200, 250, 300, 350, 400, 450, 500],
          'rfc__max_features': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]}

grid_search = GridSearchCV(pipe, param_grid=params)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=12)
    
fitted_search = grid_search.fit(X_train, y_train)

In [ ]:
fitted_search.best_params_

In [ ]:
fitted_search.best_score_

With a best accuracy score of around 76% achieved - I will train the random forest model now. I will also be adding min_samples_leaf=10 semi-arbitrarily in order to limit overfitting.

In [ ]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.33, random_state=12)

mm_scaler = MinMaxScaler()
mm_scaler.fit_transform(X_train)
mm_scaler.transform(X_validation)

rfc = RandomForestClassifier(n_estimators=200, max_depth=9, max_features=0.2, min_samples_leaf=10)

rfc.fit(X_train, y_train)

print(f"Train score: {rfc.score(X_train, y_train)}")
print(f"Validation score: {rfc.score(X_validation, y_validation)}")

In [ ]:
from sklearn.inspection import permutation_importance

# Code retrieved and modified from sklearn documentation
result = permutation_importance(
    rfc, X_train, y_train, n_repeats=10, random_state=42, n_jobs=2
)

sorted_importances_idx = result.importances_mean.argsort()
importances = pd.DataFrame(
    result.importances[sorted_importances_idx].T,
    columns=X.columns[sorted_importances_idx],
)

importance = importances.T
importance[10] = np.mean(importance, axis=1)
importance_t10 = importance.sort_values(by=10, ascending=False).head(10).T

plt.style.use('default')
plt.figure(figsize=(15, 6))
ax = importance_t10.plot.box(vert=False, whis=10)
ax.set_title("Permutation Importances (train set)")
ax.axvline(x=0, color="k", linestyle="--")
ax.set_xlabel("Decrease in accuracy score")
ax.set_ylabel("Features")
plt.savefig(fname='C:/Users/imdan/downloads/data/feature_importance1.jpeg')

In [ ]:
importance_b10 = importance.sort_values(by=10).head(10).T

ax = importance_b10.plot.box(vert=False, whis=10)
ax.set_title("Permutation Importances (train set)")
ax.axvline(x=0, color="k", linestyle="--")
ax.set_xlabel("Decrease in accuracy score")
ax.figure.tight_layout()

In [ ]:
# Checking if any features had 0 impact (training set)
importance.loc[importance[10] == 0]

I also want to check feature importance for the validation set as well - keeping in mind that the accuracy score was about 75%.

In [ ]:
result = permutation_importance(
    rfc, X_validation, y_validation, n_repeats=10, random_state=42, n_jobs=2
)

sorted_importances_idx = result.importances_mean.argsort()
importances = pd.DataFrame(
    result.importances[sorted_importances_idx].T,
    columns=X.columns[sorted_importances_idx],
)

importance = importances.T
importance[10] = np.mean(importance, axis=1)
importance_t10 = importance.sort_values(by=10, ascending=False).head(10).T

ax = importance_t10.plot.box(vert=False, whis=10)
ax.set_title("Permutation Importances (test set)")
ax.axvline(x=0, color="k", linestyle="--")
ax.set_xlabel("Decrease in accuracy score")
ax.figure.tight_layout()

In [ ]:
importance_b10 = importance.sort_values(by=10).head(10).T

ax = importance_b10.plot.box(vert=False, whis=10)
ax.set_title("Permutation Importances (test set)")
ax.axvline(x=0, color="k", linestyle="--")
ax.set_xlabel("Decrease in accuracy score")
ax.figure.tight_layout()

In [ ]:
# Checking if any features had 0 impact (training set)
importance.loc[importance[10] == 0]

Looking mostly at the training set, it was interesting that height wasn't the most predictive as screen_assist was just higher. It is a bit difficult to interpret why that is but it may be that since there are multiple clusters/groups of "big" players now and one way that they are being differentiated is through screen assists.

More interesting features that were also importance were the following:
- 2FGM_%UAST (% of 2pt field goals that were unassisted by another player)
- %PTS_FBREAK (% of points that were from fast breaks)

On the other hand, there were also some features that adversely affected accuracy scores such as OREB%, 3FGM_%AST, etc... 

Lastly, triple doubles did not affect model accuracy at all which is not surprising since they don't occur very frequently; when they do, likely random chance played a factor as well.

# Cluster Analysis

In [ ]:
# Examine Cluster 0
print(grouped_kmeans7['Cluster 0'].sort_values(ascending=False).head(15))
print()
print(grouped_kmeans7['Cluster 0'].sort_values(ascending=True).head(15))

In [ ]:
df_kmeans7 = df.copy()
df_kmeans7['kmeans_label'] = kmeans_labels

In [ ]:
df_kmeans7.loc[df_kmeans7['kmeans_label'] == 0].head(50)

In [ ]:
print(ranked_kmeans7.loc[ranked_kmeans7['Cluster 0'] == 7.0, 'Cluster 0'])
print()
print(ranked_kmeans7.loc[ranked_kmeans7['Cluster 0'] == 1.0, 'Cluster 0'])

Cluster 0 key characteristics:
- Excellent shot creation (2FGM_%UAST, 3FGM_%AUST, PULLUP_PTS, PULLUP_FG%))
- High scoring (PTS, FT%)
- 2nd best passing cluster (AST, AST%, AST/TO, AST_RATIO)
- Inefficiency (FG%, TS%)
- 3 level scoring (DRIVE_PTS, %PTS_MidRange, 16-24FGM, 8-16_FG%, 3P%)
- High turnover (TOV)
- Bad inside-defensive stats (BLK, REB, CONTEST_2PT)
- High usage (USG%)
- Small in size (Height, Weight)

The label that I will use to describe cluster 0: Offensive-Minded guards.

Notable examples: 
- Stephen Curry (2021)
- Shai Gilgeous-Alexander (2020-2022)
- C.J McCollum (2020-2022)
- Jordan Poole (2020-2022)
- Terry Rozier (2020-2022)

In [ ]:
# Examine Cluster 1
print(grouped_kmeans7['Cluster 1'].sort_values(ascending=False).head(15))
print()
print(grouped_kmeans7['Cluster 1'].sort_values(ascending=True).head(15))

In [ ]:
df_kmeans7.loc[df_kmeans7['kmeans_label'] == 1].head(50)

In [ ]:
print(ranked_kmeans7.loc[ranked_kmeans7['Cluster 1'] == 6.0, 'Cluster 1'])
print()
print(ranked_kmeans7.loc[ranked_kmeans7['Cluster 1'] == 3.0, 'Cluster 1'])

Cluster 1 Characteristics:
- Efficiency (FG%, TS%, PIE, <8FT_FG%)
- Second biggest group
- Plays inside offensively (PAINTTOUCH_PTS, POSTTOUCH_PTS, ELBOWTOUCH_PTS, %PTS_PAINT, PAINT_PTS)
- High CONTEST_3PT, PACE, AST% despite size


Label 1 players: Mobile Bigs

Notable Players:
- Karl Anthony Towns (2021-2022)
- Anthony Davis (2020-2022)
- Kristaps Porzingis (2020-2022)
- Christian Wood (2020-2022)
- John Collins (2020-2022)

In [ ]:
# Examine Cluster 2
print(grouped_kmeans7['Cluster 2'].sort_values(ascending=False).head(15))
print()
print(grouped_kmeans7['Cluster 2'].sort_values(ascending=True).head(15))

In [ ]:
df_kmeans7.loc[df_kmeans7['kmeans_label'] == 2].head(50)

In [ ]:
print(ranked_kmeans7.loc[ranked_kmeans7['Cluster 2'] == 7.0, 'Cluster 2'])
print()
print(ranked_kmeans7.loc[ranked_kmeans7['Cluster 2'] == 2.0, 'Cluster 2'])

Cluster 2 Characteristics:
- Fastest-paced players (PACE, FASTB_PTS, %PTS_FASTB, %PTS_TO)
- Outside scoring (3PA, %TEAM_3PM, 3P%, 3FGM_%UAST, CATCHSHOOT_FG%, CATCHSHOOT_PTS)
- 3 level scoring (16-24FGM, DRIVE_PTS)
- Turnover-heavy (TO_RATIO, OPP_PTS_OFF_TOV)
- Medium size
- Lack of inside scoring (%PTS_PAINT, PAINT_PTS, 2ND_PTS, PF_DRAWN)

Cluster 2 Players: Offensive-Minded Perimeter Wings
Notable players: Devin Booker (2020-2022), Zach Lavine (2020-2022), Paul George (2020-2022), Bradley Beal (2020-2021), Anthony Edwards (2021-2022)

In [ ]:
# Examine Cluster 3
print(grouped_kmeans7['Cluster 3'].sort_values(ascending=False).head(15))
print()
print(grouped_kmeans7['Cluster 3'].sort_values(ascending=True).head(15))

In [ ]:
df_kmeans7.loc[df_kmeans7['kmeans_label'] == 3].head(50)

In [ ]:
print(ranked_kmeans7.loc[ranked_kmeans7['Cluster 3'] == 5.0, 'Cluster 3'])
print()
print(ranked_kmeans7.loc[ranked_kmeans7['Cluster 3'] == 3.0, 'Cluster 3'])

Cluster 3 Characteristics:
- Great defensive stats (DEFRTG, CONTEST_3PT)
- This group is a bit more unique where they don't rank particularly amazingly/terribly in many categories and instaed is more middle-of-the-pack for a large number of statistics.
- wing/big size (3rd biggest group of players) -> will label as power forwards

Cluster 3: Versatile Power Forwards

Notable players
- Giannis Antetokounmpo (2020-2022)
- LeBron James (2022)
- Pascal Siakam (2020-2022)
- Julius Randle (2020-2022)
- Carmelo Anthony (2020-2022)

In [ ]:
# Examine Cluster 4
print(grouped_kmeans7['Cluster 4'].sort_values(ascending=False).head(15))
print()
print(grouped_kmeans7['Cluster 4'].sort_values(ascending=True).head(15))

In [ ]:
df_kmeans7.loc[df_kmeans7['kmeans_label'] == 4].head(50)

In [ ]:
print(ranked_kmeans7.loc[ranked_kmeans7['Cluster 4'] == 7.0, 'Cluster 4'])
print()
print(ranked_kmeans7.loc[ranked_kmeans7['Cluster 4'] == 1.0, 'Cluster 4'])

Cluster 4 Characteristics:
- Efficient (FG%, TS%)
- Biggest Players
- Worst shot-creation (unassisted fgs)
- Best "traditional bigs" stats (BLK, REB, 2ND_PTS, %PTS_FT, %PTS_BLK)
- Lack of outside scoring (3PA, 3P%, %PTS_MidRange)

Cluster 4 players: Stationary Interior Bigs


Notable Players:
- Joel Embiid (2020-2021)
- Nikola Jokic (2020-2022)
- Domantas Sabonis (2020-2022)
- Jonas Valanciunas (2020-2022)
- Bam Adebayo (2020-2021)

In [ ]:
# Examine Cluster 5
print(grouped_kmeans7['Cluster 5'].sort_values(ascending=False).head(15))
print()
print(grouped_kmeans7['Cluster 5'].sort_values(ascending=True).head(15))

In [ ]:
df_kmeans7.loc[df_kmeans7['kmeans_label'] == 5].head(50)

In [ ]:
print(ranked_kmeans7.loc[ranked_kmeans7['Cluster 5'] == 6.0, 'Cluster 5'])
print()
print(ranked_kmeans7.loc[ranked_kmeans7['Cluster 5'] == 2.0, 'Cluster 5'])

Cluster 5 Characteristics:
- Highest CATCHSHOOT_PTS (also 3FGM_%AST)
- Fast-paced play (PACE, %PTS_FASTB, %PTS_TO)
- Lowest usage group (USG%, TOV)
- Low scoring (PTS, %TEAM_PTS)
- Second highest CONTEST_3PT
- Wing-sized

Cluster 5: 3-D Wings

Notable Players:
- Jayson Tatum (2020-2022)
- Kevin Durant (2022)
- Jaylen Brown (2020-2021)
- Jimmy Butler (2020, 2022)
- Klay Thompson (2022)

In [ ]:
# Examine Cluster 6
print(grouped_kmeans7['Cluster 6'].sort_values(ascending=False).head(15))
print()
print(grouped_kmeans7['Cluster 6'].sort_values(ascending=True).head(15))

In [ ]:
df_kmeans7.loc[df_kmeans7['kmeans_label'] == 6].head(50)

In [ ]:
print(ranked_kmeans7.loc[ranked_kmeans7['Cluster 6'] == 7.0, 'Cluster 6'])
print()
print(ranked_kmeans7.loc[ranked_kmeans7['Cluster 6'] == 2.0, 'Cluster 6'])

Cluster 6 Characteristics:
- Highest scoring group (PTS)
- Highest assists and turnovers (AST, TOV, AST%, AST/TO, AST_RATIO)
- Highest usage (USG%, POSS)
- Smallest player group
- Best shot creation (unassisted FGM, PULLUP_PTS, PULLUP_FG%)
- Best hustle stats (CHARGES_DRAWN, LOOSE_RECOVERED_TOTAL, STL)
- Best 3-level scoring (DRIVE_PTS, CATCHSHOOT_FG%, PULLUP_PTS, %PTS_MidRange)

Cluster 6: Floor Generals

Notable players
- Luka Doncic (2020-2021)
- Trae Young (2020-2022)
- Ja Morant (2020-2022)
- Donovan Mitchell (2021-2022)
- James Harden (2020-2022)
- Chris Paul (2020-2022)

In [ ]:
df_kmeans7.to_csv('C:/Users/imdan/downloads/data/df_kmeans7.csv')

In [ ]:
df_kmeans7.loc[df_kmeans7['kmeans_label'] == 0, 'kmeans_label'] = 'Offensive-Minded Guards'
df_kmeans7.loc[df_kmeans7['kmeans_label'] == 1, 'kmeans_label'] = 'Mobile Bigs'
df_kmeans7.loc[df_kmeans7['kmeans_label'] == 2, 'kmeans_label'] = 'Offensive-Minded Perimeter Wings'
df_kmeans7.loc[df_kmeans7['kmeans_label'] == 3, 'kmeans_label'] = 'Versatile Power Forwards'
df_kmeans7.loc[df_kmeans7['kmeans_label'] == 4, 'kmeans_label'] = 'Stationary Interior Bigs'
df_kmeans7.loc[df_kmeans7['kmeans_label'] == 5, 'kmeans_label'] = '3 and D Wings'
df_kmeans7.loc[df_kmeans7['kmeans_label'] == 6, 'kmeans_label'] = 'Floor Generals'

df_kmeans7.to_csv('C:/Users/imdan/downloads/data/df_kmeans7_final.csv')

One thing to note is that there were a few other models (GMM and Agglomerative) that were experimented with but the silhouette scores did not improve, so they were not examined more deeply -> you can check notebook number 3 to see the results of those other models.

# Recommender

The next thing that I want to do is create a simple recommender system using a KNN model. I will use the KNeighborsClassifier estimator using the labels from the clustering process and optimize for accuracy score. From there, I can create a recommender system based on the nearest neighbors.

In [ ]:
df.head()

In [ ]:
# Create copy of X for KNN

X_knn = X.copy()
X_knn['CLUSTER'] = kmeans_labels
X_knn.head(10)

In [ ]:
X_knn.loc[X_knn['CLUSTER'] == 0, 'CLUSTER'] = 'CLUSTER 0'
X_knn.loc[X_knn['CLUSTER'] == 1, 'CLUSTER'] = 'CLUSTER 1'
X_knn.loc[X_knn['CLUSTER'] == 2, 'CLUSTER'] = 'CLUSTER 2'
X_knn.loc[X_knn['CLUSTER'] == 3, 'CLUSTER'] = 'CLUSTER 3'
X_knn.loc[X_knn['CLUSTER'] == 4, 'CLUSTER'] = 'CLUSTER 4'
X_knn.loc[X_knn['CLUSTER'] == 5, 'CLUSTER'] = 'CLUSTER 5'
X_knn.loc[X_knn['CLUSTER'] == 6, 'CLUSTER'] = 'CLUSTER 6'

In [ ]:
# Features and target
X = X_knn.drop('CLUSTER', axis=1)
y = X_knn['CLUSTER']

I will start with a baseline model just to get a sense of the accuracy score.

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [ ]:
X_train = mm_scaler.fit_transform(X_train)
X_test = mm_scaler.transform(X_test)

In [ ]:
pca = PCA(n_components=0.75)
pca.fit(X_train)

X_train_PCA = pca.transform(X_train)
X_test_PCA = pca.transform(X_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


neighbors = range(1, 300, 2)

train_acc = []
test_acc = []

for n in neighbors: 
    #Instantiate and Fit
    KNN_model = KNeighborsClassifier(n_neighbors=n, metric='cosine')
    KNN_model.fit(X_train, y_train)
    
    
    #Score the model
    train_accuracy = KNN_model.score(X_train, y_train)
    test_accuracy = KNN_model.score(X_test, y_test)
    
    
    #Append my accuracy
    train_acc.append(train_accuracy)
    test_acc.append(test_accuracy)

In [ ]:
#plot the graph
plt.figure(figsize=(10,7))
plt.plot(neighbors, test_acc, color="red", label="test")
plt.plot(neighbors, train_acc, color="blue", label="train")
plt.ylabel("Accuracy Score")
plt.xlabel("Number of neighbors")
plt.title("KNN Graph")
plt.legend()
plt.show()

We got an accuracy score close to 65% for our testing set. I will now implement a gridsearch in order to optimize hyperparameters and try to get the accuracy score as high as possible.

In [ ]:
X = X_knn.drop('CLUSTER', axis=1)
y = X_knn['CLUSTER']

In [ ]:
y

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn import set_config

estimators = [('normalise', MinMaxScaler()),
              ('reduce_dim', PCA()),
              ('knn', KNeighborsClassifier(n_jobs=-1))]

pipe = Pipeline(estimators)

#visualize the pipeline
set_config(display ='diagram')
pipe

In [ ]:
from sklearn.model_selection import GridSearchCV
from tempfile import mkdtemp
cachedir = mkdtemp()

pipe = Pipeline(estimators, memory = cachedir)

params = {'normalise': [MinMaxScaler(), StandardScaler(), RobustScaler()],
          'knn__weights': ['uniform','distance'],
          'knn__n_neighbors': [10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40], 
          'knn__algorithm': ['auto', 'brute'],
          'knn__metric': ['minkowski', 'cosine'],
          'reduce_dim__n_components': [.7, .75, .8, .85, .9]}

grid_search = GridSearchCV(pipe, param_grid=params)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=12)
    
fitted_search = grid_search.fit(X_train, y_train)

In [ ]:
fitted_search.best_params_

In [ ]:
fitted_search.best_score_

Accounting for all the significant hyperparameters, the highest score achieved was 0.61 which isn't amazing. Before moving forward, I will try again using LDA as dimensionality reduction.

In [ ]:
X = X_knn.drop('CLUSTER', axis=1)
y = X_knn['CLUSTER']

In [ ]:
estimators = [('reduce_dim', LDA(solver='eigen')), 
              ('knn', KNeighborsClassifier(metric='cosine'))]

pipe = Pipeline(estimators)

#visualize the pipeline
set_config(display ='diagram')
pipe

In [ ]:
cachedir = mkdtemp()

pipe = Pipeline(estimators, memory = cachedir)

params = {'knn__weights': ['uniform','distance'], 
          'knn__algorithm': ['auto', 'brute'],
          'knn__metric': ['minkowski', 'cosine'],
          'knn__n_neighbors': [10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40],
          'reduce_dim__shrinkage': [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009]}

grid_search = GridSearchCV(pipe, param_grid=params)

X = X_knn.drop('CLUSTER', axis=1)
y = df['POSITION']

X_train, X_test, y_train, y_test = train_test_split(X_lda, y, test_size=0.33, random_state=12)
    
fitted_search = grid_search.fit(X_train, y_train)

In [ ]:
fitted_search.best_params_

In [ ]:
fitted_search.best_score_

With the parameters shown above, I was able to get a score of 0.78 which is significantly better than using PCA again. I will create the recommender system with the parameters above.

In [ ]:
# Fit model
X = X_knn.drop('CLUSTER', axis=1)
y = X_knn['CLUSTER']

lda = LDA(solver='eigen', shrinkage=0.001)
lda_fit = lda.fit(X,y)
X_lda = lda_fit.transform(X)

knn = KNeighborsClassifier(n_neighbors=18)

knn_fit = knn.fit(X_lda, y)

In [ ]:
knn_fit.score(X_lda, y)

In [ ]:
X0 = X.reset_index()

In [ ]:
X_final = pd.DataFrame(data=X_lda, index=X0['index'], columns = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6'])

In [ ]:
knn_fit.kneighbors(X_final.loc[X_final.index == 'JordanPoole|2022'], n_neighbors=5)[1]

In [ ]:
knn_fit.kneighbors(X_final.loc[X_final.index == 'JordanPoole|2022'], n_neighbors=5)[0][0][4]

In [ ]:
def player_recommend(player, n=5):
    players_list = []
    distances = []
    for i in range(n):
        players_list.append(knn_fit.kneighbors(X_final.loc[X_final.index == player], n_neighbors=n)[1][0][i])
        distances.append(knn_fit.kneighbors(X_final.loc[X_final.index == player], n_neighbors=n)[0][0][i])
        
    for player in players_list:
        print(f"player: {X0.iloc[player][0]}")
    for distance in distances:
        print(f"distances: {distance}")

In [ ]:
player_recommend('GiannisAntetokounmpo|2022', n=10)

To compare my algorithm with another, I will create a simple recommender system using cosine similarity here.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
player_index = X_practice[X_practice['index'] == 'GiannisAntetokounmpo|2022'].index

In [ ]:
similarities = cosine_similarity(X, dense_output=False)

In [ ]:
sim_df = pd.DataFrame({'PLAYER':X_practice['index'], 
                       'similarity': np.array(similarities[player_index, :]).squeeze()})

In [ ]:
# Return the top 10 most similar movies
sim_df.sort_values(by='similarity', ascending=False).head(100)

With this simple recommender system, we can say that the 2 recommenders output quite different results. There are limitations to both recommender systems. This will be explored further in the analysis report (same github repo).